In [1]:
just_checking_integrity=False

In [2]:
import gc
import pandas as pd
import scipy.sparse as sps
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
URM_test=sps.load_npz("./drive/MyDrive/Recommendation_system/dataset/processed_data/URM_test_final.npz")

In [5]:
URM_test.shape

(4439967, 23690)

In [6]:
np.ravel(URM_test.sum(axis=1)).shape

(4439967,)

In [7]:
Session_ids=np.argwhere(np.ravel(URM_test.sum(axis=1)))

In [8]:
is_sorted = lambda a: np.all(a[:-1] <= a[1:])

In [9]:
is_sorted(Session_ids)

True

In [10]:
dataset=None
dataset_max=None
#["UCF","ICBF","ICF","Graph","TopPop100","TopPop50","TopPop15"] # "UCF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW", "ICBF_W"
for file in ["UCF","ICBF","Graph","ICF"]:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"./drive/MyDrive/Recommendation_system/dataset/candidates/traditional_recs/final/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


del df_feats
del df_max

0
loading UCF
15960976
loaded UCF
0
loading ICBF
14855475
loaded ICBF
0
loading Graph
8321415
loaded Graph
0
loading ICF
17771428
loaded ICF


In [11]:
ids=np.sort(dataset["Session_Id"].unique())
map_ids=pd.DataFrame()
map_ids["Session_Id"]=ids
if just_checking_integrity:
    map_ids["Real_Id"]=Session_ids[:len(ids)]
else:
    map_ids["Real_Id"]=Session_ids


In [12]:
dataset_max=dataset_max.merge(map_ids,on="Session_Id",how="left")
del dataset_max["Session_Id"]
dataset_max.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [13]:
dataset=dataset.merge(map_ids,on="Session_Id",how="left")
del dataset["Session_Id"]
dataset.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [14]:
import ast
submission_df=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/GRU/final/candidates.csv")
submission_df["score"]=submission_df["score"].map(ast.literal_eval)
submission_df["code"]=submission_df["code"].map(ast.literal_eval)
submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
submission_df = submission_df.explode(['code','score', 'rank'])
submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [15]:
import ast
submission_df_tr=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/Transformer/final/candidates.csv")
submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [16]:
import ast
submission_df_LSTM=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/LSTM/final/candidates.csv")
submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [17]:
dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
del submission_df
dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
del submission_df_tr
dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")


In [18]:
dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")
del dataset_max

In [19]:
dataset.fillna(0,inplace=True)

In [20]:
gc.collect()

374

In [21]:
dataset_macro_features = pd.read_csv('./drive/MyDrive/Recommendation_system/dataset/processed_data/macro_feats_NN_final.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')

In [22]:
# dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

In [23]:
dataset.head()

,Item_ID,Score_UCF,Score_ICBF,Score_Graph,Score_ICF,Session_Id,score_gru,rank_gru,score_transformer,rank_transformer,...,Max_Score_ICF,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,11,0.035145,0.115033,0.006554,0.311263,61,0.000000,0.0,0.001203,92.0,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,19,0.018681,0.000000,0.000000,0.000000,61,0.000000,0.0,0.000000,0.0,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,23,0.039035,0.072503,0.007389,0.325638,61,0.000000,0.0,0.000000,0.0,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,25,0.025481,0.081094,0.004554,0.006163,61,0.000000,0.0,0.000000,0.0,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,69,0.056301,0.085104,0.011284,0.338065,61,0.001834,22.0,0.002854,37.0,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [24]:
gc.collect()

0

In [25]:
dataset=dataset.sort_values("Session_Id")

In [26]:
gc.collect()

0

In [27]:
import lightgbm

In [28]:
k=10
result = []

In [29]:
for index in range(k):
    model = lightgbm.Booster(model_file=f'./drive/MyDrive/Recommendation_system/rankers/model_{index}.txt')
    result.append(model.predict(dataset.drop(["Session_Id","Item_ID"],axis=1)))

result = np.array(result)
result

array([[-0.55220689, -2.36018468, -2.19452709, ..., -1.9855451 ,
        -2.43447857, -2.18499499],
       [-0.50231822, -2.16243441, -2.19989522, ..., -1.87587463,
        -2.46630845, -2.14867063],
       [-0.37256123, -2.01998066, -2.07803124, ..., -1.80859688,
        -2.28142422, -1.99521131],
       ...,
       [-0.47056326, -2.35105571, -2.09684909, ..., -1.86761277,
        -2.31220415, -2.07518788],
       [-0.31732299, -2.199534  , -2.34060991, ..., -2.01021069,
        -2.47387731, -2.16483859],
       [-0.38472636, -2.16116834, -2.12094488, ..., -1.73514321,
        -2.24084376, -1.97149649]])

In [30]:
dataset["score"]=np.mean(result ,axis=0)
# dataset["score"]=np.median(result ,axis=0)

In [31]:
dataset=dataset[["Session_Id","Item_ID","score"]]
gc.collect()

60

In [32]:
dataset.head()

,Session_Id,Item_ID,score
323,61,4758,-0.432461
324,61,4787,-2.196734
325,61,4806,-2.159022
326,61,4846,-3.389919
15961034,61,1660,-3.045822


In [33]:
dataset=dataset.groupby(['Session_Id']).agg(list).reset_index()

In [34]:
def get_indices(x):
    x=np.array(x)
    indices_unsorted=np.argpartition(x, -100)[-100:]
    x=x[indices_unsorted]
    indices=indices_unsorted[np.argsort(-x)]
    return indices

In [35]:
dataset["indices"]=dataset["score"].apply(lambda x: get_indices(x))

In [36]:
dataset.iloc[0]

Session_Id                                                   61
Item_ID       [4758, 4787, 4806, 4846, 1660, 1663, 1695, 416...
score         [-0.4324611770773802, -2.1967343943704574, -2....
indices       [597, 397, 280, 28, 57, 491, 231, 298, 251, 23...
Name: 0, dtype: object

In [37]:
dataset.iloc[0]["Item_ID"][536]

3422

In [38]:
def get_items(x):
    items=np.array(x[0])
    return items[x[1]]

In [39]:
dataset["predictions"]=dataset[["Item_ID","indices"]].apply(get_items,axis=1)

In [40]:
dataset=dataset[["Session_Id","predictions"]]
dataset["rank"]=dataset.apply(lambda x:tuple([rank for rank in range(1, 101)]) ,axis=1)

In [41]:
dataset

,Session_Id,predictions,rank
0,61,"[1045, 292, 3586, 4574, 1563, 3414, 3581, 143,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"[3564, 3778, 4148, 3705, 4097, 143, 19, 3290, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"[4148, 4146, 2330, 1936, 3326, 2704, 3827, 165...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"[4202, 3338, 2769, 4990, 16, 2448, 1516, 3026,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"[3691, 3555, 2899, 3441, 2970, 3599, 2471, 465...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439646,"[270, 1468, 3424, 772, 3656, 191, 2720, 3618, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439648,"[4145, 3901, 3223, 4586, 3550, 3110, 3505, 19,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439675,"[4865, 3166, 4966, 603, 819, 3116, 779, 1906, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439868,"[519, 1972, 4740, 2285, 292, 3414, 4220, 750, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [42]:
dataset["predictions"]=dataset["predictions"].apply(tuple)

In [43]:
dataset=dataset[["Session_Id","predictions","rank"]]

In [44]:
dataset.columns=["session_id","code","rank"]

In [45]:
dataset.head()

,session_id,code,rank
0,61,"(1045, 292, 3586, 4574, 1563, 3414, 3581, 143,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"(3564, 3778, 4148, 3705, 4097, 143, 19, 3290, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"(4148, 4146, 2330, 1936, 3326, 2704, 3827, 165...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"(4202, 3338, 2769, 4990, 16, 2448, 1516, 3026,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"(3691, 3555, 2899, 3441, 2970, 3599, 2471, 465...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [46]:
submission_df = dataset.explode(['code', 'rank']).astype(int)

In [47]:
mapping= pd.read_csv('./drive/MyDrive/Recommendation_system/dataset/processed_data/map.csv')
submission_df=submission_df.merge(mapping,on="code",how="left")
del submission_df["code"]
submission_df=submission_df[["session_id","item_id","rank"]]

In [48]:
print(len(submission_df))

5000000


In [49]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000000 entries, 0 to 4999999
Data columns (total 3 columns):
 #   Column      Dtype
---  ------      -----
 0   session_id  int64
 1   item_id     int64
 2   rank        int64
dtypes: int64(3)
memory usage: 152.6 MB


In [50]:
submission_df.isna().sum()

session_id    0
item_id       0
rank          0
dtype: int64

In [51]:
submission_df.to_csv(
    f'./drive/MyDrive/Recommendation_system/rankers/final/final_sub.csv',
    index=False
    )